0) **Imports**

In [1]:
import os, sys
from timeit import default_timer as timer
import dolfin as df # Fenics : dolfin + ufl + FIAT + ...
import numpy as np

# fetricks is a set of utilitary functions to facilitate our lives
from fetricks.mechanics.elasticity_conversions import youngPoisson2lame
from fetricks.fenics.la.wrapper_solvers import local_project_metadata, local_project

ModuleNotFoundError: No module named 'dolfin'

1) **Consititutive behaviour Definition**

In [3]:
metric = {'YOUNG_MODULUS': 100.0,
           'POISSON_RATIO': 0.3}
lamb, mu = youngPoisson2lame(metric['POISSON_RATIO'], metric['YOUNG_MODULUS']) 

sigma_law = lambda u: lamb*df.div(u)*df.Identity(2) + mu*(df.grad(u) + df.grad(u).T)

NameError: name 'youngPoisson2lame' is not defined

2) **Mesh**  

In [ ]:
Nx =  100 # x10
Ny =  30 # x10
Lx = 2.0
Ly = 0.5
mesh = df.RectangleMesh(df.Point(0.0,0.0) , df.Point(Lx,Ly), Nx, Ny, 'left/right');
df.plot(mesh);

3) **Mesh regions** 

In [ ]:
leftBnd = df.CompiledSubDomain('near(x[0], 0.0) && on_boundary')
rightBnd = df.CompiledSubDomain('near(x[0], Lx) && on_boundary', Lx=Lx)

clampedBndFlag = 1
loadBndFlag = 2
boundary_markers = df.MeshFunction("size_t", mesh, dim=1, value=0)
leftBnd.mark(boundary_markers, clampedBndFlag)
rightBnd.mark(boundary_markers, loadBndFlag)

dx = df.Measure('dx', domain=mesh)
ds = df.Measure('ds', domain=mesh, subdomain_data=boundary_markers)

4) **Spaces**

In [ ]:
Uh = df.VectorFunctionSpace(mesh, "CG", 1) # Equivalent to CG
bcL = df.DirichletBC(Uh, df.Constant((0.0, 0.0)), boundary_markers, clampedBndFlag)
# bcL = df.DirichletBC(Uh.sub(0), df.Constant((0.0, 0.0)), boundary_markers, clampedBndFlag)

5) **Variational Formulation**: <br>

Strong format: 
$$
\begin{cases}
div \sigma = 0 \quad \text{in} \, \Omega \\
u = 0 \quad \text{on} \, \Gamma_1 \\
\varepsilon = \nabla^s u \quad \text{in} \, \Omega \\
\sigma  = \hat{\sigma}(\varepsilon) \quad \text{in} \, \Omega \\
\sigma  n  = t \quad \text{on} \, \Gamma_2 \\
\end{cases}
$$ 

Weak format: find $u \in U = \{ w \in H^1(\Omega) ; w|_{\Gamma_1} = 0 \}$ s.t.
$$
a(u, v) = b(v) \quad \forall v \in U,
$$
with
$$
\begin{cases}
a(u, v) = \int_{\Omega} \sigma(u) \cdot \nabla^s v \, dx \\
b(v) = \int_{\Gamma_2} t \cdot v \, ds \\
\end{cases}
$$

Galerkin method : find $u_h \in U_h \subset U$ s.t.
$$
a(u_h, v_h) = b(v_h) \quad \forall v_h \in U_h,
$$


In [ ]:
ty = -0.1
traction = df.Constant((0.0, ty))

In [ ]:
u = df.TrialFunction(Uh) 
v = df.TestFunction(Uh)
a = df.inner(sigma_law(u),  df.grad(v))*dx
b = df.inner(traction,v)*ds(loadBndFlag)

6) **Solving**

In [ ]:
uh = df.Function(Uh)
df.solve(a == b, uh, bcs = [bcL])
print("Norm L2: ", df.assemble(df.inner(uh,uh)*dx))

7) **Playing around**

a) *Mandel's Notation*

In [ ]:
Id_mandel_df = df.as_vector([1.0, 1.0, 0.0])
halfsqrt2 = 0.5*np.sqrt(2)

def tensor2mandel(X):
    return df.as_vector([X[0,0], X[1,1], halfsqrt2*(X[0,1] + X[1,0])])

def symgrad_mandel(v): # it was shown somehow to have better performance than doing it explicity
    return tensor2mandel(df.sym(df.grad(v)))

sigma_law = lambda w: lamb*df.div(w)*Id_mandel_df + 2*mu*symgrad_mandel(w)

#Celas = df.as_tensor( [[lamb + 2*mu, lamb, 0], [lamb, lamb + 2*mu, 0], [0, 0, 2*mu]] )
#sigma_law = lambda w: df.dot(Celas, symgrad_mandel(w))

a = df.inner(sigma_law(u),  symgrad_mandel(v))*dx

b) *Other solvers*; (Test : Comment assembling of Method 2 and Method 3) 

In [ ]:
# Method 1 : Direct, little lower-level
uh = df.Function(Uh)
problem = df.LinearVariationalProblem(a, b, uh, bcs = [bcL])

start = timer()
solver = df.LinearVariationalSolver(problem) # mumps, petsc, ...
solver.parameters["linear_solver"] = "superlu" # mumps, petsc,
solver.solve()
end = timer()
                  
print("Time spent: ", end - start)
print("Norm L2: ", df.assemble(df.inner(uh,uh)*dx))


# Method 2 : Iterative, low-level
uh = df.Function(Uh)
start = timer()
A, F = df.assemble_system(a, b, bcs = [bcL])

solver = df.PETScKrylovSolver('gmres','hypre_amg') 
solver.parameters["relative_tolerance"] = 1e-5
solver.parameters["absolute_tolerance"] = 1e-6
solver.parameters["error_on_nonconvergence"] = False
solver.parameters["maximum_iterations"] = 1000
solver.parameters["monitor_convergence"] = True
solver.set_operator(A)
solver.solve(uh.vector(), F)   
end = timer()


print("Time spent: ", end - start)
print("Norm L2: ", df.assemble(df.inner(uh,uh)*dx))

# Method 3 : Direct, low-level 
uh = df.Function(Uh)
F_ = df.PETScVector()
A_ = df.PETScMatrix()
solver_ = df.PETScLUSolver(A_)

start = timer()
df.assemble(a, tensor = A_) 
bcL.apply(A_)
df.assemble(b, tensor = F_)
bcL.apply(F_)

solver_.solve(uh.vector(), F_)    
end = timer()

print("Time spent: ", end - start)
print("Norm L2: ", df.assemble(df.inner(uh,uh)*dx))

c) *Pos-Processing*

In [ ]:
import fetricks.fenics.postprocessing.wrapper_io as iofe
from fetricks.fenics.la.wrapper_solvers import local_project

iofe.exportXDMF_gen("bar_vtk.xdmf", fields={'vertex': [uh] })

#os.system("paraview bar_vtk.xdmf")

d) *Nonlinear solver* <br> Let us suppose 
\begin{align} 
\min_{u \in U} \left ( J(u):=\int_{\Omega} \psi(u) dx - \Pi_{ext}(u) \right) \\
F(u; v) = \delta J(u;v) = 0 \quad \forall v \in V ,
\end{align} 
<br>
with
$$
\psi(\varepsilon(u)) = \frac{\lambda}{2}
( tr {\varepsilon}^2 + \frac{\alpha}{2} tr {\varepsilon}^4) + 
\mu ( |\varepsilon|^2  + \frac{\alpha}{2} |\varepsilon|^4).
$$

In [ ]:
alpha = 10e4 # set to zero

def psi_e(e):
    tr_e = e[0] + e[1]
    e2 = df.inner(e,e)
    
    return (0.5*lamb*(tr_e**2 + 0.5*alpha*tr_e**4) +
           mu*(e2 + 0.5*alpha*e2**2))

psi = lambda w: psi_e(symgrad_mandel(w))

du = df.TrialFunction(Uh)            # Incremental displacement
v  = df.TestFunction(Uh)             # Test function
uh  = df.Function(Uh)                 # Displacement from previous iteration

Pi = psi(uh)*dx - df.inner(traction,uh)*ds(loadBndFlag)

F = df.derivative(Pi, uh, v)
Jac = df.derivative(F, uh, du) # Optional

# Compute solution
start = timer()
df.solve(F == 0, uh, bcL, J = Jac)
end = timer()

print("Time spent: ", end - start)
print("Norm L2: ", df.assemble(df.inner(uh,uh)*dx))
# Ref : 0.00046895042573293067

In [ ]:
def sigma_law(w):    
    e = symgrad_mandel(w)
    e = df.variable(e)
    return df.diff(psi_e(e),e)

# Alternative formulation: hand-coded
def sigma_law_handcoded(w):
    e = symgrad_mandel(w)
    tr_e = e[0] + e[1]
    e2 = df.inner(e,e)
    
    return lamb*(1 + alpha*tr_e**2)*tr_e*Id_mandel_df  + 2*mu*(1.0 + alpha*e2)*e



e) Quadrature Spaces

In [ ]:
import warnings
from ffc.quadrature.deprecation import QuadratureRepresentationDeprecationWarning
warnings.simplefilter("once", QuadratureRepresentationDeprecationWarning)

df.parameters["form_compiler"]["representation"] = 'quadrature'
# df.parameters["form_compiler"]["optimize"] = True
# df.parameters["form_compiler"]["cpp_optimize"] = True
# df.parameters["form_compiler"]["cpp_optimize_flags"] = "-O3"

# Degree of the stress : test for 0 and 1
deg = 0

Qe = df.VectorElement("Quadrature", mesh.ufl_cell(), degree= deg + 1 , dim = 3, quad_scheme='default')
sh0_Q = df.FunctionSpace(mesh, Qe ) # for stress

sh0 = df.VectorFunctionSpace(mesh , 'DG', degree = deg, dim = 3)

metadata = {"quadrature_degree": deg + 1, "quadrature_scheme": "default"}
dxm = df.Measure( 'dx', mesh, metadata=metadata)

start = timer()
sigh_Q = local_project_metadata(sigma_law(uh), sh0_Q, metadata)
end = timer()
print("Time spent local project Quadrature: ", end - start)


start = timer()
sigh_ = local_project(sigma_law_handcoded(uh), sh0)
# sigh_ = local_project(sigma_law(uh), sh0)
end = timer()
print("Time spent local project DG: ", end - start)


print("norm (DG) : " , np.linalg.norm(sigh_.vector().get_local()) )
print("norm (Q) : " , np.linalg.norm(sigh_Q.vector().get_local()) )
print("error : " , np.linalg.norm(sigh_Q.vector().get_local() - sigh_.vector().get_local()) )
print("num cells: ", mesh.num_cells(), "sh0 dim:", sh0.dim(), "sh0_Q dim:", sh0_Q.dim())


f) Export dataset

In [ ]:
from ddfenics.dd.ddfunction import DDFunction
epsh = DDFunction(sigh_.function_space())
sigh = DDFunction(sigh_.function_space())

sigh.update(sigh_)
epsh.update(symgrad_mandel(uh))

sigh.rename('sigma', '')
uh.rename('u', '')

print(np.min(epsh.data(), axis = 0))
print(np.max(epsh.data(), axis = 0))

data = np.concatenate((epsh.data(), sigh.data()), axis = 1)
np.savetxt('database_ref.txt', data, header = '1.0 \n%d 2 3 3'%data.shape[0], comments = '', fmt='%.8e', )